In [1]:
import os
project_dir = '/Users/mumtaz/Documents/projects/zavmo/zavmo-api/zavmo'
os.chdir(project_dir)
os.getcwd()

'/Users/mumtaz/Documents/projects/zavmo/zavmo-api/zavmo'

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import json
from IPython.display import Markdown
from openai import OpenAI
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [4]:
from helpers.functions import get_prompt, get_yaml_data

In [5]:
conf_data       = get_yaml_data('discuss')
agent_keys      = ['name', 'description', 'instructions', 'examples', 'completion_condition', 'next_stage', 'next_stage_description']
prompt_context  = {k:v for k,v in conf_data.items() if k in agent_keys}

In [6]:
system_message = get_prompt('probe.md', context=prompt_context,prompt_dir="assets/prompts")

In [24]:
message_history = json.loads(open(r'/Users/mumtaz/Documents/projects/zavmo/message_history.json', 'r').read())

In [25]:
from pydantic import BaseModel, Field
from typing import List, Dict
from helpers._types import (
    Agent,
    StrictTool,
    Result,
)
from helpers.swarm import fetch_agent_response

In [26]:
Markdown(system_message)

You are Zavmo, a friendly and engaging personal learning companion. Zavmo uses a unique 4D learning process: Discover, Discuss, Deliver, and Demonstrate.

You are currently guiding the learner through the 'Discussion' stage, on the Zavmo platform.  You should highlight your role as a dedicated learning partner—someone learners can talk to, ask questions, and explore ideas with at any time. Focus on creating an experience that feels natural, personal, and engaging. Through this approach, you engage learners in meaningful discussions, adapt to their interests and learning styles, and provide a highly interactive, personalized experience.

### Guidelines

• **Engagement & Personalization**: Use a friendly and adaptable tone that reflects the user’s preferences. Personalize conversations with their name and use emojis where appropriate for a more relatable, human-like interaction.

• **Guided Learning**: Guide users step-by-step through Zavmo’s 4-step learning process, ensuring clarity, focus, and comprehension at each stage. Adjust explanations based on user understanding.

• **Motivation & Feedback**: Keep learners engaged and motivated with consistent positive reinforcement, timely feedback, and encouragement to help them stay on track and inspired to continue.

• **Interaction Design**: Structure the interaction flow with short, one message at a time to encourage thoughtful responses. Avoid overwhelming the user by pacing the conversation appropriately and keeping your messages concise.

• **Emotional Sensitivity & Adaptability**: Be responsive to emotional cues in the user’s language. Adapt your tone and pacing based on user sentiment. Offer additional support, such as simplified explanations or accommodations, for neurodiverse learners or those showing signs of confusion or frustration.

• **Language**: Use British English for all your interactions.


### Context for this stage:
The Discussion stage is the second stage of the 4-D Learning Journey. In this stage, Zavmo will discuss with the learner about their preferred learning style, available study time, and design a curriculum based on the TNA assessment data and OFQUAL Units.


### Instructions:
1. First, ask the learner about their preferred learning style, It can be one of the following:
   - Case Study Discussions: Interactive cases tailored to learner's goals
   - Socratic Dialogue: Guided questioning to enhance critical thinking
   - Problem-Based Learning: Real-world scenarios for problem-solving
   - Mentoring Conversations: One-on-one guided discussions
   - Storytelling Learning: Industry-relevant narrative discussions
  Present these options (along with brief descriptions) to the learner and ask them to choose one. These are the only learning options available. If the learner expresses a preference for a non-conversational format, politely explain that the learning experience is fully conversation-based for the time being.
2. Second, ask the learner's about their available study time per week. 3. Only after the user has provided their learning style, and timeline, use the `update_discussion_data` tool with instructions to update the learner's learning style and timeline. 4. Immediately after the discussion data is updated, a priority data for curriculum generation will be provided. Use the `Curriculum` tool to generate a curriculum of upto 10 modules, following the below two instructions closely specific to curriculum generation:
    a. Include upto 5 modules addressing Assessment Areas with learning outcomes based on identified gaps. If no gaps are specified, base outcomes on the core Assessment Area concepts.
    b. Prioritize upto 5 OFQUAL Units as standalone modules in the curriculum. Each OFQUAL Unit must be implemented as a separate, complete module with dedicated learning outcomes aligned to qualification standards.
5. You must present the generated curriculum in a table format to the learner and ask them to review it, explaining how it addresses their Assessment Areas and OFQUAL Units. 6. Ask the learner if they would like to make any changes to the curriculum while maintaining alignment with OFQUAL standards. 7. If the learner is satisfied, transfer to the delivery stage using the `transfer_to_delivery_stage` tool.


### Example interactions to guide your role:
['How do you prefer to learn, for example, role-play, storytelling, case study discussions, or scenario-based learning?', 'Awesome choice, [learner_name]! How many hours per week can you dedicate to learning? ⏳', "Based on our discussion and TNA results, we've crafted something truly exciting for you to progress and fill the knowledge gaps. Let me show you how these map to professional qualifications. ", "Here's how we have structured your curriculum:\n## Introduction to Business Management\n| **Subject:** | Business Management | | **Level:** | Intermediate |\n### Modules\n| **Module**                           | **Duration** | **Learning Outcomes**                             | **Lessons**                                  | |--------------------------------------|--------------|--------------------------------------------------------------------------------------------------| | **[Assessment Area]**                | 10 hours     | [Gaps determined will become learning outcomes]   | [Based on Gaps determined design lessons]    | | **[Assessment Area]**                | 8 hours      | [Gaps determined will become learning outcomes]   | [Based on Gaps determined design lessons]    | | **[OFQUAL Unit Data Based Title]**   | 8 hours      | [OFQUAL Unit based learning outcomes]             | [OFQUAL Unit based lessons]                  | | **[OFQUAL Unit Data Based Title]**   | 8 hours      | [OFQUAL Unit based learning outcomes]             | [OFQUAL Unit based lessons]                  | | **[OFQUAL Unit Data Based Title]**   | 8 hours      | [OFQUAL Unit based learning outcomes]             | [OFQUAL Unit based lessons]                  |\nWhat do you think of this plan? Does it fit your available time? 📚", "Absolutely! Let's make adjustments based on your preferences. 📅", "Noted! We'll refine the curriculum as needed. 🎯", "If you're happy with the curriculum and timeline, I will transfer you to the next stage where we will provide you with learning materials and resources tailored to your needs."]

### Your role is complete when:
Learner approves their learning objectives, learning outcomes, and curriculum.


### Handoff instructions:
Once your task is complete, seamlessly hand off to the next stage: Delivery. The next stage involves:
The Delivery stage is the third stage of the 4-D Learning Journey. In this stage, Zavmo will provide the learner with learning materials and resources tailored to their needs. In the Delivery stage, Zavmo will personalise each lesson based on the curriculum, timeline, and modules discussed in the discussion stage, as well as the interest areas and learning style.


In [27]:
class Module(BaseModel):
    title: str = Field(description="The title of the module. It can be an Assessment Area or OFQUAL Unit")
    learning_outcomes: List[str] = Field(description="List upto 5 learning outcomes.")
    lessons: List[str] = Field(description="List of lessons in this module based on the learning outcomes.")
    duration: int = Field(description="The total duration of the module in hours")

class Curriculum(BaseModel):
    """Generate a detailed curriculum for the learner based on the Assessment Areas and corresponding OFQUAL Units shared."""
    title: str = Field(description="The title of the curriculum")
    subject: str = Field(description="The main subject area of the curriculum.")
    level: str = Field(description="The difficulty level of the curriculum (e.g., beginner, intermediate, advanced)")
    prerequisites: List[str] = Field(description="Any prerequisites needed to undertake this curriculum")
    modules: List[Module] = Field(description="List upto 10 or more modules majorly designed on Assessment Areas and OFQUAL Units data shared. Include 1-2 modules on learner's interest areas as well.")

    def execute(self, context: Dict):
        return Result(value=str(self.model_dump()), context=context)    

discuss_agent = Agent(
    name="Discussion",
    id="discuss",
    instructions=system_message,
    functions=[
        Curriculum
    ],
    parallel_tool_calls=False,
    tool_choice='auto',
    model="gpt-4o"
)

In [31]:
completion = fetch_agent_response(agent=discuss_agent, context={}, history = message_history)

In [32]:
message = completion.choices[0].message

In [33]:
curriculum = json.loads(message.tool_calls[0].function.arguments)['modules']

In [23]:
for module in curriculum:
    print(module['title'])

Victim and Witness Interviews
Principles of Planning and Reporting
Supervising Investigators
Principles of Private Investigation
Theory, Policy, and Legislation in Investigations
Interview Techniques and Procedures
Surveillance and Information Gathering
Maintaining Security and Confidentiality
Investigative Health and Safety
Serious and Complex Investigations


In [30]:
message_history[-1]['content'] = """Discussion data updated successfully for ds_testuser_33@test.com
        
**Timeline**: 8
**Learning Style**: Case Study Discussions

The following data sources are being used to inform the personalized curriculum design:
1. All TNA Assessment Data
2. All OFQUAL Units Data


- **All TNA Assessment Data provided below must be used for Curriculum Generation**:

**Assessment Area:** Victim and witness interviews in serious and complex investigations
**Learner's Report:** Alex provided a detailed reflection on a recent case where cognitive interviewing techniques were used, including rapport building, open-ended questions, and active listening, leading to a successful investigation outcome.
**Gaps Determined:** []

**Assessment Area:** Principles of Planning and Reporting Investigations
**Learner's Report:** In a recent investigation, Alex led the planning phase conducting a risk assessment and established objectives. He provided evidence-based reports, highlighted key findings and insights, facilitating informed stakeholder decisions.
**Gaps Determined:** []

**Assessment Area:** Supervise police investigations and investigators
**Learner's Report:** In a recent investigation supervised by Alex, proactive leadership with clear objectives and open communication led to resolving challenges like conflicting priorities. Alex's supervision promoted consistent progress and enhanced investigative quality.
**Gaps Determined:** []

**Assessment Area:** Principles of private investigation
**Learner's Report:** In a recent private investigation, Alex used OSINT, surveillance, and interviews to gather evidence. Detailed documentation and maintaining a clear chain of custody ensured evidence integrity.
**Gaps Determined:** ['Incorporating more digital forensics for quicker data retrieval.', 'Enhancing collaboration with subject matter experts for deeper analysis.']

**Assessment Area:** Understand theory, policy, legislation and process for conducting serious and complex investigations
**Learner's Report:** In a complex investigation involving financial fraud, Alex applied relevant legislation and adhered to procedural guidelines, ensuring a lawful and thorough investigation that supported a successful prosecution.
**Gaps Determined:** []



- **All OFQUAL Units Data provided below must be used for Curriculum Generation**:

**OFQUAL ID: 610/3782/5 (Unit: D/6475):**
1. Understand issues associated with victim and witness care during a complex investigation.
  - Explain how Adverse Childhood Experiences (ACE) may impact on individuals.
  - Evaluate the impact of trauma on victims and witnesses involved in complex offending.

**OFQUAL ID: 600/3345/9 (Unit: K/616/0757):**
1. Know how to establish the objectives and ongoing feasibility of an investigation
  - Determine the purpose of different investigations
  - Assess what information is needed to meet different investigational requirements
  - Identify criteria for establishing the feasibility of an investigation
  - State the appropriate legislation that should be followed when carrying out investigations
  - Identify possible consequences of carrying out an investigation
  - Explain how to establish collaborative relationships with all involved
2. Know how to deal with actual or potential conflicts of interest
  - State what options are available to deal with conflicts of interest in an investigation
  - Explain the possible consequences of not dealing effectively with conflicts of interest
3. Know how to report and present the findings of an investigation
  - Describe the different types of investigation reports
  - Describe the requirements of investigation reports in terms of meeting client and legal needs

**OFQUAL ID: 601/7799/8 (Unit: CI105):**
1. Understand the requirements for supervising police investigations and investigators
  - Summarise legislation, codes of practice and guidelines for supervising and conducting investigations
  - Summarise policies and procedures for supervising and conducting investigations
  - Explain the procedures for disclosing information in accordance with national crime recording standards
  - Explain how to select investigators according to their skills and experience in relation to the investigation concerned
  - Summarise the risk assessment processes associated with conducting police investigations
2. Be able to supervise police investigations in line with legal and organisational requirements
  - Prepare information for investigations
  - Evaluate the progress of investigations
  - Take action to support investigators if necessary
  - Confirm that individuals are dealt with in an ethical manner, recognising their needs with respect to equality and diversity
  - Confirm case files are submitted
  - Verify post-charge procedures are completed in line with operational requirements
  - Confirm the investigation has been finalised in accordance with national crime recording standards
  - Complete a lessons learnt log that reflects the outcomes of investigations
  - Share lessons learnt with relevant others
3. Be able to supervise and support police investigators
  - Clarify roles and responsibilities to people involved in investigations
  - Confirm that investigators follow risk assessment procedures associated with conducting police investigations
  - Verify that investigators develop effective investigation plans
  - Provide support to investigators where required to ensure that all lines of enquiry are dealt with
  - Confirm that investigators document information in accordance with policy and legislation
  - Confirm that investigators pass on information to the appropriate person or department

**OFQUAL ID: 603/2950/6 (Unit: Y/616/8904):**
1. Be able to establish the objectives and ongoing requirement of an investigation
  - Determine the purpose of various investigations
  - Analyse and assess what information is required to meet the requirements of an investigation
  - Identify criteria for establishing the requirement/demand of an investigation
  - Summarise the appropriate legislation that should be undertaken when conducting investigations
  - Establish the possible consequences of carrying out an investigation
  - Articulate methods on establishing collaborative relationships with all key stakeholders involved
2. Understand actions to deal with conflicts of interest
  - Explain what options are available to deal with conflicts of interest within an investigation
  - Assess the consequences of not dealing effectively with conflicts of interest
3. Know how to report and present the findings of an investigation
  - Explain the different investigation formats
  - Produce investigation reports that meet client and legal requirements
4. Know how to research and analyse information in support of an investigation
  - Establish and prioritise appropriate sources of information to support an investigation
  - Evaluate the various methods for researching information
  - Explain why information and evidence should be analysed and graded
  - Summarise the importance and value of corroborating information
  - Explain the fundamentals of maintaining the continuity and integrity of evidence
  - Explain how to maintain the continuity and integrity of evidence
  - Explain the requirements of legislation relevant to searching for, obtaining and handling information, including the disposal of information not needed to support an investigation
5. Know how to interview people to gather information
  - Summarise the various formats of interviews and when they should be used
  - Describe the various interview techniques and procedures
  - Describe how to carry out effective interviews
  - Summarise the different legislations and proposed SIA Standards of Behaviour in terms of the impact on interviewing
  - Describe how information should be recorded during an interview and stored after the interview for evidence in court
6. Know how surveillance can be used to gather information
  - Explain how to prepare and plan a surveillance operation
  - Explain the difference between overt and covert surveillance techniques that comply with relevant legislation
  - Identify the various equipment that can legally be used during surveillance operations in terms of its capabilities, limitations and suitability
7. Understand how to maintain the security and confidentiality of information gathered
  - Explain why the security and confidentiality of information must be maintained
  - Describe how the security and confidentiality of information must be maintained
8. Understand relevant Health and Safety issues
  - Explain the types of situations where the personal health and safety of the investigator may be at risk
  - Explain how to avoid or minimise personal health and safety risks

**OFQUAL ID: 610/3782/5 (Unit: D/6473):**
1. Understand the difference between 'volume and priority investigations' and 'serious and complex' investigations.
  - Explain the difference between 'volume and priority' investigations and 'serious and complex' investigations.
  - Explain factors that will escalate a volume and priority investigation to serious and complex.
2. Understand the legislation, policy and process associated with serious and complex investigations.
  - Summarise investigative legislation applicable to serious and complex investigations.
  - Explain considerations around covert methods and their use in serious and complex investigations."""

In [18]:
print(message_history[-1]['content'])

Discussion data updated successfully for ds_testuser_33@test.com
        
**Timeline**: 8
**Learning Style**: Case Study Discussions

The following data sources are being used to inform the personalized curriculum design:
1. All TNA Assessment Data
2. All OFQUAL Units data


- **All TNA Assessment Data provided below must be used for Curriculum Generation**:

**Assessment Area:** Victim and witness interviews in serious and complex investigations
**Learner's Report:** Alex provided a detailed reflection on a recent case where cognitive interviewing techniques were used, including rapport building, open-ended questions, and active listening, leading to a successful investigation outcome.
**Gaps Determined:** []

**Assessment Area:** Principles of Planning and Reporting Investigations
**Learner's Report:** In a recent investigation, Alex led the planning phase conducting a risk assessment and established objectives. He provided evidence-based reports, highlighted key findings and insight